<a href="https://colab.research.google.com/github/under71/assist/blob/main/%EB%B0%95%EB%B0%B0%EC%98%81%20RAG_08_LangChain_CES2025_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 벡터 DB 사용 RAG 구성

In [ ]:

# 연관 라이브러리

# #  The issue is with the update of httpx to 0.28. (proxies)
!pip install httpx==0.27.2

!pip install python-dotenv
# !pip install faiss
!pip install langchain
!pip install openai
!pip install tiktoken
# for CPU
# !apt install libomp-dev
# !pip install faiss

# for GPU
!pip install faiss-gpu


ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [ ]:
import urllib.request
import os


# txt 경로
#txt_url = "https://raw.githubusercontent.com/under71/assist/refs/heads/main/ces2025_ko.txt"
#txt_url = "https://raw.githubusercontent.com/under71/assist/refs/heads/main/ces2025_en.txt"
txt_url = "https://raw.githubusercontent.com/under71/assist/refs/heads/main/ces2025_jp.txt"
txt_dir = "./"
#txt_file =  'ces2025_ko.txt'
#txt_file =  'ces2025_en.txt'
txt_file =  'ces2025_jp.txt'
txt_path = os.path.join(txt_dir, txt_file)
urllib.request.urlretrieve(txt_url, txt_path)

('./ces2025_jp.txt', <http.client.HTTPMessage at 0x7f5061c88c10>)

In [ ]:
!cat ces2025_ko.txt
!cat ces2025_jp.txt
!cat ces2025_en.txt

젠슨 황 엔비디아 창업자가 CES 2025 개막 기조연설을 했다. 현장은 설렘과 기대감, 호기심으로 가득 찼다. 자신감과 카리스마 넘치는 연설은 순식간에 청중을 빨아들였다. 
연설과 함께 필자는 CES 2025 혁신상 심사위원으로서 그의 기조연설에 나타난 핵심 주제를 분석해봤다.황의 기조연설은 AI와 컴퓨팅 혁신, 산업 AI와 디지털 트윈, 자율주행과 합성 데이터, 휴머노이드 로봇과 일반 로봇공학, AI 에이전트로 요약됐다.
◇ AI와 컴퓨팅의 혁신, 새로운 미래 연다
젠슨 황은 GPU가 현대 컴퓨팅의 패러다임 전환을 이끌고 있다고 진단했다. 기존의 CPU 중심 컴퓨팅에서 GPU 기반 가속 컴퓨팅으로 이동하며, AI가 컴퓨팅 인프라의 중심축으로 자리 잡고 있다는 분석이다.
그는 엔비디아의 쿠다(CUDA)는 프로그래밍 가능한 GPU를 통해 복잡한 AI 알고리즘을 효율적으로 처리할 수 있도록 함으로써 다양한 AI 연구와 응용을 가능하게 했다고 설명했다.
또한 AI가 텍스트, 이미지, 음성 등을 종합적으로 이해하고 생성할 수 있는 능력을 가지면서 AI가 새로운 차원으로 진화하고 있다.
◇ AI와 디지털 트윈, 산업혁신 이끈다
젠슨 황은 디지털 트윈(Digital Twin)이 AI와 물리적 세계를 연결해 산업혁신을 일이킬 것이라고 전망했다. 
그는 엔비디아의 옴니버스(Omniverse) 플랫폼이 물리적 환경을 디지털로 시뮬레이션해주고 코스모스(Cosmos)가 AI와 물리적 데이터를 결합해 시뮬레이션을 고도화하는 역할을 한다고 설명했다. 황은 이를 통해 제조, 물류, 에너지 산업에서 최적의 프로세스를 설계해 산업을 혁신할 수 있다고 단언했다.
디지털 트윈은 모든 물리적 환경을 디지털로 재현하고 최적화할 수 있는 강력한 도구로 AI의 진화는 제조와 물류 산업의 미래를 변화시킬 전망이다.
◇ AI 합성 데이터, 자율주행 안전시대 연다 
젠슨 황은 자율주행 기술의 발전에서 합성 데이터의 중요성을 특히 강조했다. 자율주행차의 AI 모델은 합성 데이터를 통해 훈련되며, 이 데이

In [ ]:
from getpass import getpass

OPENAI_API_KEY = getpass("OpenAI API Key : ")


OpenAI API Key : ··········


In [ ]:
!pip install -U langchain-community

In [ ]:
pip install faiss-cpu  ## FAISS cpu 버젼 설치


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 58.9 MB/s eta 0:00:00


In [ ]:
try:   # FAISS 설치 확인
    import faiss
    print("FAISS successfully imported.")
except ImportError as e:
    print(f"FAISS import failed: {e}")

FAISS successfully imported.


In [ ]:

import dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


# loads .env file with your OPENAI_API_KEY
dotenv.load_dotenv()

# Doc Info from 젠슨황의 CES 2025 기조연결

# loader = TextLoader("ces2025_ko.txt",encoding="CP949")
# loader = TextLoader("ces2025_ko.txt",encoding="euc-kr")
loader = TextLoader("ces2025_ko.txt",encoding="utf-8")
# loader = TextLoader("ces2025_en.txt",encoding="utf-8")

documents = loader.load()

# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
print("len(docs) == ", len(docs))

faissIndex = FAISS.from_documents(docs, OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY))
print("faissIndex == ", faissIndex )
# OpenAIEmbeddings()
info_idx = "faiss_ces2025_ko" # 파일이름  txt
#info_idx = "faiss_ces2025_en" # 파일이름  txt
#info_idx = "faiss_ces2025_jp" # 파일이름  txt

faissIndex.save_local(info_idx)

len(docs) ==  1
faissIndex ==  <langchain_community.vectorstores.faiss.FAISS object at 0x7f50614aec50>


In [ ]:
# 연관 자료탐색
# print("docs[0] == ", docs[0])

# question = "질문 1: 젠슨 황의 CES 2025 기조연설의 주요 주제는 무엇인가요?"
# question = "질문 2: 디지털 트윈(Digital Twin)이 산업 혁신에 어떻게 기여한다고 설명되었나요? 한줄로 답해줘"
# question = "질문 3: 젠슨 황은 AI가 새로운 노동 계층을 어떻게 창출한다고 전망했나요?"
# question = "앞으로 ai에서 가장 중요한것은 무엇인가요?"

query = question
retriever = faissIndex.as_retriever(search_type="similarity", search_kwargs={"k":5})
relevant_docs =  retriever.get_relevant_documents(query)

num_docs = len(relevant_docs)
print("len(relevant_docs) == ", num_docs)
for i in range(num_docs) :
 print("relevant_docs[", i , "] == ", relevant_docs[i])



len(relevant_docs) ==  1
relevant_docs[ 0 ] ==  page_content='젠슨 황 엔비디아 창업자가 CES 2025 개막 기조연설을 했다. 현장은 설렘과 기대감, 호기심으로 가득 찼다. 자신감과 카리스마 넘치는 연설은 순식간에 청중을 빨아들였다. 
연설과 함께 필자는 CES 2025 혁신상 심사위원으로서 그의 기조연설에 나타난 핵심 주제를 분석해봤다.황의 기조연설은 AI와 컴퓨팅 혁신, 산업 AI와 디지털 트윈, 자율주행과 합성 데이터, 휴머노이드 로봇과 일반 로봇공학, AI 에이전트로 요약됐다.
◇ AI와 컴퓨팅의 혁신, 새로운 미래 연다
젠슨 황은 GPU가 현대 컴퓨팅의 패러다임 전환을 이끌고 있다고 진단했다. 기존의 CPU 중심 컴퓨팅에서 GPU 기반 가속 컴퓨팅으로 이동하며, AI가 컴퓨팅 인프라의 중심축으로 자리 잡고 있다는 분석이다.
그는 엔비디아의 쿠다(CUDA)는 프로그래밍 가능한 GPU를 통해 복잡한 AI 알고리즘을 효율적으로 처리할 수 있도록 함으로써 다양한 AI 연구와 응용을 가능하게 했다고 설명했다.
또한 AI가 텍스트, 이미지, 음성 등을 종합적으로 이해하고 생성할 수 있는 능력을 가지면서 AI가 새로운 차원으로 진화하고 있다.
◇ AI와 디지털 트윈, 산업혁신 이끈다
젠슨 황은 디지털 트윈(Digital Twin)이 AI와 물리적 세계를 연결해 산업혁신을 일이킬 것이라고 전망했다. 
그는 엔비디아의 옴니버스(Omniverse) 플랫폼이 물리적 환경을 디지털로 시뮬레이션해주고 코스모스(Cosmos)가 AI와 물리적 데이터를 결합해 시뮬레이션을 고도화하는 역할을 한다고 설명했다. 황은 이를 통해 제조, 물류, 에너지 산업에서 최적의 프로세스를 설계해 산업을 혁신할 수 있다고 단언했다.
디지털 트윈은 모든 물리적 환경을 디지털로 재현하고 최적화할 수 있는 강력한 도구로 AI의 진화는 제조와 물류 산업의 미래를 변화시킬 전망이다.
◇ AI 합성 데이터, 자율주행 안전시대 연다 
젠슨 황은 자율주행 기술의 

aaa#


In [ ]:
import os, dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain import PromptTemplate

dotenv.load_dotenv()

chatbot = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(
        openai_api_key=OPENAI_API_KEY,
        temperature=0,
        # model_name="gpt-4o",
        model_name="gpt-4o-mini",
        # model_name="gpt-3.5-turbo",
        max_tokens=2000,  #max_tokens=50
        streaming=True,
    ),
    #     llm=OpenAI(
    #     openai_api_key=os.getenv("OPENAI_API_KEY"),
    #     temperature=0, model_name="text-davinci-003", max_tokens=1000  #max_tokens=50
    # ),
    chain_type="stuff",
    retriever=retriever,
    # retriever=FAISS.load_local(info_idx, OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
    #                            allow_dangerous_deserialization=True).as_retriever(search_type="similarity", search_kwargs={"k":1},)
)


# template = """
# 충청도 사투리로  상세하게 답을 해줘. {query}?
# """


# template = """
# respond as succinctly as possible like the following :
# 1. xxx
# 2. xxx
# ...
# 10. xxx

# {query}?
# """

template = """
respond as thoroughly as possible. {query}?
"""


# # 없는 정보를 사용하지 않도록 제한

/

# template = """
# {query}
# 질문에 대해서 아래에 주어진 문맥을 바탕으로 제주도 사투리로 답을 해줘.
# 문맥에 답이 없으면 '몰라용' 이라고 해줘.
# """


# template = """
# {query}
# 질문에 대해서 아래에 주어진 문맥을 바탕으로 북한 아나운서 스타일로 답을 해줘.
# 문맥에 답이 없으면 '몰라용' 이라고 해줘.
# """

# template = """
# {query}

# """

# template = """
# {query}

# Answer in Korean.
# """


prompt = PromptTemplate(
    input_variables=["query"],
    template=template,
)

Context = [{'role':'system','content':""" You’re a helpful assistant.
Please answer in a lastly questioned language."""}]
# accumulate message
# question = "what is --v"
# print(chatbot.run(
#     prompt.format(query=question)
# ))
# # --v is a parameter used to specify a specific model version in Midjourney's AI image generation tool.

In [ ]:

#######
# question = "젠승황의 주요요지는?"
question = "질문 1: 젠슨 황의 CES 2025 기조연설의 주요 주제는 무엇인가요??"
# question = "what is autopilot?"  # 오류도 발생
# question = "what is main issue of CEs 2025?"            #
# question = "人工知能自律走行とは何ですか？?"             #
# question = "what is AI Robot?"
# question = "Quels sont les principaux thèmes abordés dans le discours de Jensen Huang à l'ouverture du CES 2025 ??"            # 프랑스어질문 (젠슨 황이 CES 2025 개막 연설에서 다룬 주요 주제는 무엇인가요?)

# question = "Welche Rolle spielt die GPU-Technologie bei der Transformation moderner Computersysteme laut Jensen Huang? "   #독일어  질문 : (젠슨 황에 따르면 GPU 기술이 현대 컴퓨팅 시스템의 전환에서 어떤 역할을 하나요?)
# question = "根据黄仁勋的演讲，数字孪生技术如何推动工业创新？"           # 중국어 질문 (젠슨 황의 연설에 따르면 디지털 트윈 기술이 산업 혁신을 어떻게 촉진하나요?)





print(chatbot.run(
    prompt.format(query=question)
))

젠슨 황의 CES 2025 기조연설의 주요 주제는 다음과 같이 요약될 수 있습니다:

1. **AI와 컴퓨팅 혁신**: 젠슨 황은 GPU가 현대 컴퓨팅의 패러다임 전환을 이끌고 있으며, AI가 컴퓨팅 인프라의 중심축으로 자리 잡고 있다고 강조했습니다. 그는 엔비디아의 쿠다(CUDA) 기술을 통해 복잡한 AI 알고리즘을 효율적으로 처리할 수 있는 가능성을 설명했습니다.

2. **AI와 디지털 트윈**: 디지털 트윈이 AI와 물리적 세계를 연결하여 산업 혁신을 이끌 것이라고 전망했습니다. 엔비디아의 옴니버스(Omniverse) 플랫폼이 물리적 환경을 디지털로 시뮬레이션하고, 코스모스(Cosmos)가 AI와 물리적 데이터를 결합해 시뮬레이션을 고도화하는 역할을 한다고 설명했습니다.

3. **AI 합성 데이터와 자율주행**: 자율주행 기술의 발전에서 합성 데이터의 중요성을 강조하며, 합성 데이터가 AI 모델의 훈련에 기여하고 기술의 신뢰성을 높이는 역할을 한다고 언급했습니다.

4. **AI와 휴머노이드 로봇**: 로봇공학이 AI와 결합하여 인간과 협력하는 새로운 시대를 열 것이라고 전망했습니다. 휴머노이드 로봇이 인간의 동작을 모방하고 다양한 산업에서 활동하게 될 것이라고 설명했습니다.

5. **AI가 창출하는 새로운 노동 계층**: AI가 기존의 노동 시장을 재편하고 새로운 형태의 노동을 만들어낼 것이라고 강조했습니다. AI 기반 에이전트가 데이터 분석, 고객 서비스, 소프트웨어 개발 등에서 인간을 돕는 역할을 하게 될 것이라고 언급했습니다.

이러한 주제들은 AI가 단순한 기술의 진보를 넘어 사회와 산업의 구조를 근본적으로 변화시키는 혁신의 중심에 있음을 강조하며, AI와 인간이 함께 설계하는 미래의 가능성을 제시하고 있습니다.
